In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pandas numpy pywavelets scikit-learn plotly

In [3]:
import csv

# Chemin vers le fichier CSV
file_path = '/content/drive/MyDrive/P2M/sfmconnect.disaggregationsNXT3.csv'

# Ensemble pour stocker les équipements uniques
equipments = set()

# Lecture du fichier CSV
with open(file_path, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        # Ajouter les équipements à l'ensemble
        for i in range(4):  # Checking all 4 equipment columns
            equipment_name = row[f'equipments[{i}].equipment.name']
            if equipment_name:
                equipments.add(equipment_name)

# Afficher les équipements uniques
print("Équipements existants dans le dataset :")
for equipment in sorted(equipments):
    print(f"- {equipment}")

Équipements existants dans le dataset :
- AUTRES
- Imprimante
- Ordinateurs
- Salle serveur 2


In [4]:
import pandas as pd
import pywt
import numpy as np
from sklearn.metrics import mean_squared_error
import plotly.graph_objs as go

# Charger le dataset
file_path = '/content/drive/MyDrive/P2M/sfmconnect.disaggregationsNXT3.csv'
df = pd.read_csv(file_path)

# Extraire les lignes où le premier équipement est une imprimante
df_printer = df[df["equipments[0].equipment.name"] == "Imprimante"]

# Vérifier si on a trouvé des données
if df_printer.empty:
    raise ValueError("❌ Aucun signal trouvé pour 'Imprimante' dans equipments[0].equipment.name.")

# Extraire le signal d'énergie active correspondant
signal = df_printer["equipments[0].consumption.Active_energy"].dropna().values
timestamps = pd.to_datetime(df_printer["date"], unit='ms')  # Convertir les timestamps

# Fonction de compression avec statistiques
def compress_with_stats(signal, wavelet_name, keep_ratio=0.05):
    coeffs = pywt.wavedec(signal, wavelet=wavelet_name)
    coeff_arr, coeff_slices = pywt.coeffs_to_array(coeffs)

    threshold = np.quantile(np.abs(coeff_arr), 1 - keep_ratio)
    mask = (np.abs(coeff_arr) >= threshold)
    compressed_coeff_arr = coeff_arr * mask
    compressed_coeffs = pywt.array_to_coeffs(compressed_coeff_arr, coeff_slices, output_format='wavedec')

    reconstructed_signal = pywt.waverec(compressed_coeffs, wavelet=wavelet_name)[:len(signal)]
    rmse = np.sqrt(mean_squared_error(signal, reconstructed_signal))

    original_size = len(coeff_arr)
    transmitted_size = np.count_nonzero(mask)
    compression_ratio = transmitted_size / original_size

    return {
        'wavelet': wavelet_name,
        'reconstructed': reconstructed_signal,
        'rmse': rmse,
        'original_size': original_size,
        'transmitted_size': transmitted_size,
        'compression_ratio': compression_ratio
    }

# Liste des ondelettes à tester
wavelets = ['db8', 'sym8', 'coif5', 'bior3.5', 'haar']
results = []

# Appliquer la compression pour chaque ondelette
for w in wavelets:
    result = compress_with_stats(signal, w)
    results.append(result)

# Afficher les résultats
print("📊 Résultats de la compression sur le signal de l'imprimante :\n")
for r in results:
    print(f"➡️ {r['wavelet']}:")
    print(f"   RMSE               : {r['rmse']:.6f}")
    print(f"   Taille originale   : {r['original_size']}")
    print(f"   Données transmises : {r['transmitted_size']}")
    print(f"   Ratio de compression: {r['compression_ratio']:.2%}\n")

# Visualisation interactive avec Plotly
fig = go.Figure()

# Signal original
fig.add_trace(go.Scatter(
    x=timestamps,
    y=signal,
    mode='lines',
    name='Signal original',
    line=dict(color='black', width=2)
))

# Ajouter les signaux reconstruits
colors = ['red', 'green', 'purple', 'blue', 'orange']
for i, r in enumerate(results):
    fig.add_trace(go.Scatter(
        x=timestamps,
        y=r['reconstructed'],
        mode='lines',
        name=f"Reconstruction ({r['wavelet']})",
        line=dict(color=colors[i], dash='dash')
    ))

# Mettre à jour la mise en page
fig.update_layout(
    title="Comparaison de compression du signal de l'imprimante",
    xaxis_title="Temps",
    yaxis_title="Énergie active (kWh)",
    template="plotly_white",
    legend=dict(x=0, y=1)
)

# Afficher la figure
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [5]:
import pandas as pd
import pywt
import numpy as np
from sklearn.metrics import mean_squared_error
import plotly.graph_objs as go

# Charger le dataset
file_path = '/content/drive/MyDrive/P2M/sfmconnect.disaggregationsNXT3.csv'
df = pd.read_csv(file_path)

# Extraire les lignes où le premier équipement est une imprimante
df_printer = df[df["equipments[0].equipment.name"] == "Imprimante"]

# Vérifier si on a trouvé des données
if df_printer.empty:
    raise ValueError("❌ Aucun signal trouvé pour 'Imprimante' dans equipments[0].equipment.name.")

# Extraire le signal d'énergie active correspondant
signal = df_printer["equipments[0].consumption.Active_energy"].dropna().values
timestamps = pd.to_datetime(df_printer["date"], unit='ms')  # Convertir les timestamps

# Fonction de compression avec statistiques
def compress_with_stats(signal, wavelet_name, keep_ratio=0.05):
    coeffs = pywt.wavedec(signal, wavelet=wavelet_name)
    coeff_arr, coeff_slices = pywt.coeffs_to_array(coeffs)

    threshold = np.quantile(np.abs(coeff_arr), 1 - keep_ratio)
    mask = (np.abs(coeff_arr) >= threshold)
    compressed_coeff_arr = coeff_arr * mask
    compressed_coeffs = pywt.array_to_coeffs(compressed_coeff_arr, coeff_slices, output_format='wavedec')

    reconstructed_signal = pywt.waverec(compressed_coeffs, wavelet=wavelet_name)[:len(signal)]
    rmse = np.sqrt(mean_squared_error(signal, reconstructed_signal))

    original_size = len(coeff_arr)
    transmitted_size = np.count_nonzero(mask)
    compression_ratio = transmitted_size / original_size

    return {
        'wavelet': wavelet_name,
        'reconstructed': reconstructed_signal,
        'rmse': rmse,
        'original_size': original_size,
        'transmitted_size': transmitted_size,
        'compression_ratio': compression_ratio
    }

# Liste des ondelettes à tester
wavelets = ['db8', 'sym8', 'coif5', 'bior3.5', 'haar']
results = []

# Appliquer la compression pour chaque ondelette
for w in wavelets:
    result = compress_with_stats(signal, w)
    results.append(result)

# Afficher les résultats
print("📊 Résultats de la compression sur le signal de l'imprimante :\n")
for r in results:
    print(f"➡️ {r['wavelet']}:")
    print(f"   RMSE               : {r['rmse']:.6f}")
    print(f"   Taille originale   : {r['original_size']}")
    print(f"   Données transmises : {r['transmitted_size']}")
    print(f"   Ratio de compression: {r['compression_ratio']:.2%}\n")

# Visualisation interactive avec Plotly
fig = go.Figure()

# Signal original
fig.add_trace(go.Scatter(
    x=timestamps,
    y=signal,
    mode='lines',
    name='Signal original',
    line=dict(color='black', width=2)
))

# Ajouter les signaux reconstruits
colors = ['red', 'green', 'purple', 'blue', 'orange']
for i, r in enumerate(results):
    fig.add_trace(go.Scatter(
        x=timestamps,
        y=r['reconstructed'],
        mode='lines',
        name=f"Reconstruction ({r['wavelet']})",
        line=dict(color=colors[i], dash='dash')
    ))

# Mettre à jour la mise en page
fig.update_layout(
    title="Comparaison de compression du signal de l'imprimante",
    xaxis_title="Temps",
    yaxis_title="Énergie active (kWh)",
    template="plotly_white",
    legend=dict(x=0, y=1)
)

# Afficher la figure
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [6]:
import pandas as pd
import pywt
import numpy as np
from sklearn.metrics import mean_squared_error
import plotly.graph_objs as go

# Charger le dataset
file_path = '/content/drive/MyDrive/P2M/sfmconnect.disaggregationsNXT3.csv'
df = pd.read_csv(file_path)

# Extraire les lignes où le premier équipement est Ordinateurs
df_computers = df[df["equipments[1].equipment.name"] == "Ordinateurs"]

# Vérifier si on a trouvé des données
if df_computers.empty:
    raise ValueError("❌ Aucun signal trouvé pour 'Ordinateurs' dans equipments[1].equipment.name.")

# Extraire le signal d'énergie active correspondant
signal = df_computers["equipments[1].consumption.Active_energy"].dropna().values
timestamps = pd.to_datetime(df_computers["date"], unit='ms')

# Fonction de compression avec statistiques
def compress_with_stats(signal, wavelet_name, keep_ratio=0.05):
    coeffs = pywt.wavedec(signal, wavelet=wavelet_name)
    coeff_arr, coeff_slices = pywt.coeffs_to_array(coeffs)

    threshold = np.quantile(np.abs(coeff_arr), 1 - keep_ratio)
    mask = (np.abs(coeff_arr) >= threshold)
    compressed_coeff_arr = coeff_arr * mask
    compressed_coeffs = pywt.array_to_coeffs(compressed_coeff_arr, coeff_slices, output_format='wavedec')

    reconstructed_signal = pywt.waverec(compressed_coeffs, wavelet=wavelet_name)[:len(signal)]
    rmse = np.sqrt(mean_squared_error(signal, reconstructed_signal))

    original_size = len(coeff_arr)
    transmitted_size = np.count_nonzero(mask)
    compression_ratio = transmitted_size / original_size

    return {
        'wavelet': wavelet_name,
        'reconstructed': reconstructed_signal,
        'rmse': rmse,
        'original_size': original_size,
        'transmitted_size': transmitted_size,
        'compression_ratio': compression_ratio
    }

# Liste des ondelettes à tester
wavelets = ['db8', 'sym8', 'coif5', 'bior3.5', 'haar']
results = []

# Appliquer la compression pour chaque ondelette
for w in wavelets:
    result = compress_with_stats(signal, w)
    results.append(result)

# Afficher les résultats
print("📊 Résultats de la compression sur le signal des ordinateurs :\n")
for r in results:
    print(f"➡️ {r['wavelet']}:")
    print(f"   RMSE               : {r['rmse']:.6f}")
    print(f"   Taille originale   : {r['original_size']}")
    print(f"   Données transmises : {r['transmitted_size']}")
    print(f"   Ratio de compression: {r['compression_ratio']:.2%}\n")

# Visualisation interactive avec Plotly
fig = go.Figure()

# Signal original
fig.add_trace(go.Scatter(
    x=timestamps,
    y=signal,
    mode='lines',
    name='Signal original',
    line=dict(color='black', width=2)
))

# Ajouter les signaux reconstruits
colors = ['red', 'green', 'purple', 'blue', 'orange']
for i, r in enumerate(results):
    fig.add_trace(go.Scatter(
        x=timestamps,
        y=r['reconstructed'],
        mode='lines',
        name=f"Reconstruction ({r['wavelet']})",
        line=dict(color=colors[i], dash='dash')
    ))

# Mettre à jour la mise en page
fig.update_layout(
    title="Comparaison de compression du signal des ordinateurs",
    xaxis_title="Temps",
    yaxis_title="Énergie active (kWh)",
    template="plotly_white",
    legend=dict(x=0, y=1)
)

# Afficher la figure
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [7]:
import pandas as pd
import pywt
import numpy as np
from sklearn.metrics import mean_squared_error
import plotly.graph_objs as go

# Charger le dataset
file_path = '/content/drive/MyDrive/P2M/sfmconnect.disaggregationsNXT3.csv'
df = pd.read_csv(file_path)

# Extraire les lignes où le quatrième équipement est Salle serveur 2
df_server = df[df["equipments[3].equipment.name"] == "Salle serveur 2"]

# Vérifier si on a trouvé des données
if df_server.empty:
    raise ValueError("❌ Aucun signal trouvé pour 'Salle serveur 2' dans equipments[3].equipment.name.")

# Extraire le signal d'énergie active correspondant
signal = df_server["equipments[3].consumption.Active_energy"].dropna().values
timestamps = pd.to_datetime(df_server["date"], unit='ms')

# Fonction de compression avec statistiques
def compress_with_stats(signal, wavelet_name, keep_ratio=0.05):
    coeffs = pywt.wavedec(signal, wavelet=wavelet_name)
    coeff_arr, coeff_slices = pywt.coeffs_to_array(coeffs)

    threshold = np.quantile(np.abs(coeff_arr), 1 - keep_ratio)
    mask = (np.abs(coeff_arr) >= threshold)
    compressed_coeff_arr = coeff_arr * mask
    compressed_coeffs = pywt.array_to_coeffs(compressed_coeff_arr, coeff_slices, output_format='wavedec')

    reconstructed_signal = pywt.waverec(compressed_coeffs, wavelet=wavelet_name)[:len(signal)]
    rmse = np.sqrt(mean_squared_error(signal, reconstructed_signal))

    original_size = len(coeff_arr)
    transmitted_size = np.count_nonzero(mask)
    compression_ratio = transmitted_size / original_size

    return {
        'wavelet': wavelet_name,
        'reconstructed': reconstructed_signal,
        'rmse': rmse,
        'original_size': original_size,
        'transmitted_size': transmitted_size,
        'compression_ratio': compression_ratio
    }

# Liste des ondelettes à tester
wavelets = ['db8', 'sym8', 'coif5', 'bior3.5', 'haar']
results = []

# Appliquer la compression pour chaque ondelette
for w in wavelets:
    result = compress_with_stats(signal, w)
    results.append(result)

# Afficher les résultats
print("📊 Résultats de la compression sur le signal de la salle serveur 2 :\n")
for r in results:
    print(f"➡️ {r['wavelet']}:")
    print(f"   RMSE               : {r['rmse']:.6f}")
    print(f"   Taille originale   : {r['original_size']}")
    print(f"   Données transmises : {r['transmitted_size']}")
    print(f"   Ratio de compression: {r['compression_ratio']:.2%}\n")

# Visualisation interactive avec Plotly
fig = go.Figure()

# Signal original
fig.add_trace(go.Scatter(
    x=timestamps,
    y=signal,
    mode='lines',
    name='Signal original',
    line=dict(color='black', width=2)
))

# Ajouter les signaux reconstruits
colors = ['red', 'green', 'purple', 'blue', 'orange']
for i, r in enumerate(results):
    fig.add_trace(go.Scatter(
        x=timestamps,
        y=r['reconstructed'],
        mode='lines',
        name=f"Reconstruction ({r['wavelet']})",
        line=dict(color=colors[i], dash='dash')
    ))

# Mettre à jour la mise en page
fig.update_layout(
    title="Comparaison de compression du signal de la salle serveur 2",
    xaxis_title="Temps",
    yaxis_title="Énergie active (kWh)",
    template="plotly_white",
    legend=dict(x=0, y=1)
)

# Afficher la figure
fig.show()

In [8]:
import pandas as pd
import pywt
import numpy as np
from sklearn.metrics import mean_squared_error
import plotly.graph_objs as go

# Charger le dataset
file_path = '/content/drive/MyDrive/P2M/sfmconnect.disaggregationsNXT3.csv'
df = pd.read_csv(file_path)

# Extraire les lignes où le troisième équipement est AUTRES
df_others = df[df["equipments[2].equipment.name"] == "AUTRES"]

# Vérifier si on a trouvé des données
if df_others.empty:
    raise ValueError("❌ Aucun signal trouvé pour 'AUTRES' dans equipments[2].equipment.name.")

# Extraire le signal d'énergie active correspondant
signal = df_others["equipments[2].consumption.Active_energy"].dropna().values
timestamps = pd.to_datetime(df_others["date"], unit='ms')

# Fonction de compression avec statistiques
def compress_with_stats(signal, wavelet_name, keep_ratio=0.05):
    coeffs = pywt.wavedec(signal, wavelet=wavelet_name)
    coeff_arr, coeff_slices = pywt.coeffs_to_array(coeffs)

    threshold = np.quantile(np.abs(coeff_arr), 1 - keep_ratio)
    mask = (np.abs(coeff_arr) >= threshold)
    compressed_coeff_arr = coeff_arr * mask
    compressed_coeffs = pywt.array_to_coeffs(compressed_coeff_arr, coeff_slices, output_format='wavedec')

    reconstructed_signal = pywt.waverec(compressed_coeffs, wavelet=wavelet_name)[:len(signal)]
    rmse = np.sqrt(mean_squared_error(signal, reconstructed_signal))

    original_size = len(coeff_arr)
    transmitted_size = np.count_nonzero(mask)
    compression_ratio = transmitted_size / original_size

    return {
        'wavelet': wavelet_name,
        'reconstructed': reconstructed_signal,
        'rmse': rmse,
        'original_size': original_size,
        'transmitted_size': transmitted_size,
        'compression_ratio': compression_ratio
    }

# Liste des ondelettes à tester
wavelets = ['db8', 'sym8', 'coif5', 'bior3.5', 'haar']
results = []

# Appliquer la compression pour chaque ondelette
for w in wavelets:
    result = compress_with_stats(signal, w)
    results.append(result)

# Afficher les résultats
print("📊 Résultats de la compression sur le signal des autres équipements :\n")
for r in results:
    print(f"➡️ {r['wavelet']}:")
    print(f"   RMSE               : {r['rmse']:.6f}")
    print(f"   Taille originale   : {r['original_size']}")
    print(f"   Données transmises : {r['transmitted_size']}")
    print(f"   Ratio de compression: {r['compression_ratio']:.2%}\n")

# Visualisation interactive avec Plotly
fig = go.Figure()

# Signal original
fig.add_trace(go.Scatter(
    x=timestamps,
    y=signal,
    mode='lines',
    name='Signal original',
    line=dict(color='black', width=2)
))

# Ajouter les signaux reconstruits
colors = ['red', 'green', 'purple', 'blue', 'orange']
for i, r in enumerate(results):
    fig.add_trace(go.Scatter(
        x=timestamps,
        y=r['reconstructed'],
        mode='lines',
        name=f"Reconstruction ({r['wavelet']})",
        line=dict(color=colors[i], dash='dash')
    ))

# Mettre à jour la mise en page
fig.update_layout(
    title="Comparaison de compression du signal des autres équipements",
    xaxis_title="Temps",
    yaxis_title="Énergie active (kWh)",
    template="plotly_white",
    legend=dict(x=0, y=1)
)

# Afficher la figure
fig.show()